## 季度调整

In [1]:
import pandas as pd

In [2]:
from zipline.pipeline.builtin import quarterly_multiplier, changed_locations

In [3]:
from pandas.tseries.offsets import QuarterEnd

In [4]:
dates = pd.date_range('2016-4-2','2018-5-2')

In [5]:
qe = QuarterEnd()

In [6]:
q_dates = pd.DatetimeIndex([qe.apply(x) for x in dates])

In [7]:
# 选取季度数子发生变化的最近四个季度
locs = changed_locations(q_dates, include_first=True)[-4:]

**说明**
+ 利润表为时期数，各科目为截止报告日期的累计数，以最近四个季度预测全年，简单方法是先将各季度数字转换为年度数，即原始数据乘以乘数，然后四个季度简单平均
+ 一季度乘数为4,2季度为2,三季度为4/3,四季度为1
+ 数据单位由万元调整到元

In [8]:
from zipline.research import run_pipeline, select_output_by
from zipline.pipeline import Pipeline

## `book_value_per_share`

In [9]:
from zipline.pipeline.builtin import book_value_per_share

In [10]:
def make_pipeline():
    return Pipeline(columns={'每股账面价值': book_value_per_share()})


result = run_pipeline(make_pipeline(), '2018-2-23', '2018-2-24')
select_output_by(result,stock_codes=['000333','600645'])

每股账面价值
2018-02-23 00:00:00+00:00 美的集团(000333)  12.639009
                          中源协和(600645)   4.242958

## `TTMSale` 尾部12个月营业总收入

In [11]:
from zipline.pipeline.builtin import TTMSale

In [12]:
def make_pipeline():
    ts = TTMSale()
    return Pipeline(
        columns={
            'TTM总收入':ts
        }, 
    )

In [13]:
result = run_pipeline(make_pipeline(), '2018-2-23', '2018-2-24')

In [14]:
select_output_by(result,stock_codes=['600645'])

,,TTM总收入
2018-02-23 00:00:00+00:00,中源协和(600645),8.389142e+08


In [15]:
pred = 8.389142e+08
actu = 87091 * 10000
diff = pred / actu - 1

print('预测与实际差异率={}%'.format(round(diff * 100,2)))

预测与实际差异率=-3.67%


In [16]:
select_output_by(result,stock_codes=['000333'])

,,TTM总收入
2018-02-23 00:00:00+00:00,美的集团(000333),2.455383e+11


In [17]:
pred = 2.455383e+11
actu = 24191890 * 10000
diff = pred / actu - 1

print('预测与实际差异率={}%'.format(round(diff * 100,2)))

预测与实际差异率=1.5%


## `TTMDividend` & `trailing_dividend_yield`
+ 尾部12个月股利及股利率

In [18]:
from zipline.pipeline.data import USEquityPricing
from zipline.pipeline.fundamentals import Fundamentals
from zipline.pipeline.builtin import TTMDividend, trailing_dividend_yield

In [19]:
def make_pipeline():
    return Pipeline(
        columns={
            '每股股利': Fundamentals.dividend.amount.latest,
            '登记日期': Fundamentals.dividend.asof_date.latest,
            '收盘价': USEquityPricing.close.latest,
            'TTM股利': TTMDividend(),
            'TTM每股股利收益率': trailing_dividend_yield()
        })

In [20]:
result = run_pipeline(make_pipeline(), '2017-5-8', '2017-5-12')

In [21]:
select_output_by(result,stock_codes=['000333'])

,,TTM每股股利收益率,TTM股利,收盘价,每股股利,登记日期
2017-05-08 00:00:00+00:00,美的集团(000333),0.030429,1.1,36.15,1.2,2016-05-05
2017-05-09 00:00:00+00:00,美的集团(000333),0.030830,1.1,35.68,1.2,2016-05-05
2017-05-10 00:00:00+00:00,美的集团(000333),0.032070,1.1,34.30,1.0,2017-05-09
2017-05-11 00:00:00+00:00,美的集团(000333),0.031294,1.1,35.15,1.0,2017-05-09
2017-05-12 00:00:00+00:00,美的集团(000333),0.030709,1.1,35.82,1.0,2017-05-09


## `earning_yield`

In [22]:
from zipline.pipeline.builtin import earning_yield

In [23]:
def make_pipeline():
    return Pipeline(
        columns={
            '稀释EPS': Fundamentals.profit_statement.A045.latest,
            '收盘价': USEquityPricing.close.latest,
            '稀释每股收益率': earning_yield(),
        })

In [24]:
result = run_pipeline(make_pipeline(), '2017-5-8', '2017-5-12')
select_output_by(result,stock_codes=['000333'])

,,收盘价,稀释EPS,稀释每股收益率
2017-05-08 00:00:00+00:00,美的集团(000333),36.15,2.28,0.063071
2017-05-09 00:00:00+00:00,美的集团(000333),35.68,2.28,0.063901
2017-05-10 00:00:00+00:00,美的集团(000333),34.30,2.28,0.066472
2017-05-11 00:00:00+00:00,美的集团(000333),35.15,2.28,0.064865
2017-05-12 00:00:00+00:00,美的集团(000333),35.82,2.28,0.063652
